In [1]:
import boto3 
client = boto3.client('glue')

In [2]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()
print(role)

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
print(bucket)

arn:aws:iam::228889150161:role/SageMakerTutorialRole


INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-228889150161


sagemaker-us-east-1-228889150161


In [10]:
script_loc = 's3://{}/glue/nyt-comments/tokenize.scala'.format(bucket)
sagemaker_jar = 's3://{}/glue/nyt-comments/jars/sagemaker-spark_2.11-spark_2.2.0-1.0.5.jar'.format(bucket)

client.create_job(Name='sagemaker-nyt-prep',
    Description='Spark ML job to create a bag-of-words vector',
    Role=role,
    Command={
        'Name':'glueetl', 
        'ScriptLocation':script_loc},
    DefaultArguments={
        '--job-language': 'scala',
        '--class': 'Tokenizer'
        '--extra-jars': sagemaker_jar,
        '--sagemaker-bucket': bucket
    })

{'Name': 'sagemaker-nyt-prep',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '29',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 01 Jun 2018 22:05:15 GMT',
   'x-amzn-requestid': 'dcd77b79-65e7-11e8-be74-79fa2354f605'},
  'HTTPStatusCode': 200,
  'RequestId': 'dcd77b79-65e7-11e8-be74-79fa2354f605',
  'RetryAttempts': 0}}

In [11]:
client.start_job_run(JobName='sagemaker-nyt-prep', AllocatedCapacity=10)

{'JobRunId': 'jr_65ef9d4b51bd131324926ca125b86cb77e48e05f2253c5d2211ed0d019ae3bf4',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '82',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 01 Jun 2018 22:08:13 GMT',
   'x-amzn-requestid': '46e152fc-65e8-11e8-a0fd-b56210d48105'},
  'HTTPStatusCode': 200,
  'RequestId': '46e152fc-65e8-11e8-a0fd-b56210d48105',
  'RetryAttempts': 0}}

In [12]:
client.get_job(JobName='test2')

{'Job': {'AllocatedCapacity': 10,
  'Command': {'Name': 'glueetl',
   'ScriptLocation': 's3://sagemaker-us-east-1-228889150161/glue/nyt-comments/gluetest'},
  'Connections': {},
  'CreatedOn': datetime.datetime(2018, 5, 31, 20, 56, 1, 125000, tzinfo=tzlocal()),
  'DefaultArguments': {'--TempDir': 's3://aws-glue-temporary-228889150161-us-east-1/root',
   '--class': 'Tokenizer',
   '--extra-jars': 's3://sagemaker-us-east-1-228889150161/glue/nyt-comments/jars/sagemaker-spark_2.11-spark_2.2.0-1.0.5.jar',
   '--job-bookmark-option': 'job-bookmark-disable',
   '--job-language': 'scala',
   'dataset.bucket': 'foo'},
  'ExecutionProperty': {'MaxConcurrentRuns': 1},
  'LastModifiedOn': datetime.datetime(2018, 6, 1, 21, 48, 9, 886000, tzinfo=tzlocal()),
  'MaxRetries': 0,
  'Name': 'test2',
  'Role': 'arn:aws:iam::228889150161:role/SageMakerTutorialRole',
  'Timeout': 2880},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '715',
   'content-type': 'applicat